<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#INCEPTION" data-toc-modified-id="INCEPTION-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>INCEPTION</a></span></li></ul></div>

In [1]:
import torch
from torchsummary import summary
import torch.nn.functional as F
from torch import nn

## INCEPTION
<img src="../images/inception.jpg" />

In [2]:
class Inception_block(nn.Module):
    def __init__(self,in_channels,c1,c2,c3,c4,**kwargs):
        super().__init__(**kwargs)
        # Path 1 is a single 1 x 1 convolutional layer
        self.p1_1=nn.Conv2d(in_channels,out_channels=c1,kernel_size=1)
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3
        # convolutional layer
        self.p2_1=nn.Conv2d(in_channels,out_channels=c2[0],kernel_size=1)
        self.p2_2=nn.Conv2d(c2[0],c2[1],kernel_size=3,padding=1)
        # Path 2 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer
        self.p3_1=nn.Conv2d(in_channels,out_channels=c3[0],kernel_size=1)
        self.p3_2=nn.Conv2d(c3[0],c3[1],kernel_size=5,padding=2)
        # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1=nn.MaxPool2d(kernel_size=3,padding=1,stride=1)
        self.p4_2=nn.Conv2d(in_channels,out_channels=c4,kernel_size=1)
        self.relu=nn.ReLU()
    def forward(self,x):
        p1=self.relu(self.p1_1(x))
        p2=self.relu(self.p2_2(self.relu(self.p2_1(x))))
        p3=self.relu(self.p3_2(self.relu(self.p3_1(x))))
        p4=self.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1,p2,p3,p4),axis=1)

GoogLeNet uses a stack of a total of 9 inception blocks and global average pooling to generate its estimates. Maximum pooling between inception blocks reduced the
dimensionality. The first part is identical to AlexNet and LeNet, the stack of blocks is inherited
from VGG and the global average pooling avoids a stack of fully-connected layers at the end. The
architecture is depicted below
<img src="../images/inception1.jpg" />
<img src="../images/inception2.jpg" />

In [3]:
inception=nn.Sequential(
        nn.Conv2d(1,64,kernel_size=7,stride=2,padding=1),nn.ReLU(),
              nn.MaxPool2d(kernel_size=3,padding=1,stride=2),
    
              nn.Conv2d(64,64,kernel_size=1),nn.ReLU(),
              nn.Conv2d(64,192,kernel_size=3,padding=1),nn.ReLU(),
              nn.MaxPool2d(kernel_size=3,padding=1,stride=2),
    
              # inception(3a)
              Inception_block(in_channels=192,c1=64,c2=(96,128),c3=(16,32),c4=32),
               # inception(3b)
              Inception_block(in_channels=256,c1=128,c2=(128,192),c3=(32,96),c4=64),
              nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
    
              # inception(4a)
              Inception_block(in_channels=480,c1=192,c2=(96,208),c3=(16,48),c4=64),
              # inception(4b)
              Inception_block(in_channels=512,c1=160,c2=(112,224),c3=(24,64),c4=64),
              # inception(4c)
              Inception_block(in_channels=512,c1=128,c2=(128,256),c3=(24,64),c4=64),
              # inception(4d)
              Inception_block(512,112,(144,288),(32,64),64),
              # inception(4e)
              Inception_block(528,256,(160,320),(32,128),128),
              nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
    
              # inception(5a)
              Inception_block(832,256, (160, 320), (32, 128), 128),
              # inception(5b)
              Inception_block(832,384, (192, 384), (48, 128), 128),
              nn.AdaptiveAvgPool2d((1,1)),
              nn.Flatten(),

              nn.Dropout(0.4),
              nn.Linear(1024,10),
              nn.Softmax(dim=1)
              )
    

In [4]:
X = torch.randn(size=(1, 1, 96, 96))
for layer in inception:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2d output shape:	 torch.Size([1, 64, 46, 46])
ReLU output shape:	 torch.Size([1, 64, 46, 46])
MaxPool2d output shape:	 torch.Size([1, 64, 23, 23])
Conv2d output shape:	 torch.Size([1, 64, 23, 23])
ReLU output shape:	 torch.Size([1, 64, 23, 23])
Conv2d output shape:	 torch.Size([1, 192, 23, 23])
ReLU output shape:	 torch.Size([1, 192, 23, 23])
MaxPool2d output shape:	 torch.Size([1, 192, 12, 12])
Inception_block output shape:	 torch.Size([1, 256, 12, 12])
Inception_block output shape:	 torch.Size([1, 480, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 480, 6, 6])
Inception_block output shape:	 torch.Size([1, 512, 6, 6])
Inception_block output shape:	 torch.Size([1, 512, 6, 6])
Inception_block output shape:	 torch.Size([1, 512, 6, 6])
Inception_block output shape:	 torch.Size([1, 528, 6, 6])
Inception_block output shape:	 torch.Size([1, 832, 6, 6])
MaxPool2d output shape:	 torch.Size([1, 832, 3, 3])
Inception_block output shape:	 torch.Size([1, 832, 3, 3])
Inception_block output sh

In [5]:
X = torch.randn(size=(1, 1, 96, 96))
summary(inception,X)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 46, 46]          3,200
├─ReLU: 1-2                              [-1, 64, 46, 46]          --
├─MaxPool2d: 1-3                         [-1, 64, 23, 23]          --
├─Conv2d: 1-4                            [-1, 64, 23, 23]          4,160
├─ReLU: 1-5                              [-1, 64, 23, 23]          --
├─Conv2d: 1-6                            [-1, 192, 23, 23]         110,784
├─ReLU: 1-7                              [-1, 192, 23, 23]         --
├─MaxPool2d: 1-8                         [-1, 192, 12, 12]         --
├─Inception_block: 1-9                   [-1, 256, 12, 12]         --
|    └─Conv2d: 2-1                       [-1, 64, 12, 12]          12,352
|    └─ReLU: 2-2                         [-1, 64, 12, 12]          --
|    └─Conv2d: 2-3                       [-1, 96, 12, 12]          18,528
|    └─ReLU: 2-4                         [-1, 96, 12, 12]         

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 46, 46]          3,200
├─ReLU: 1-2                              [-1, 64, 46, 46]          --
├─MaxPool2d: 1-3                         [-1, 64, 23, 23]          --
├─Conv2d: 1-4                            [-1, 64, 23, 23]          4,160
├─ReLU: 1-5                              [-1, 64, 23, 23]          --
├─Conv2d: 1-6                            [-1, 192, 23, 23]         110,784
├─ReLU: 1-7                              [-1, 192, 23, 23]         --
├─MaxPool2d: 1-8                         [-1, 192, 12, 12]         --
├─Inception_block: 1-9                   [-1, 256, 12, 12]         --
|    └─Conv2d: 2-1                       [-1, 64, 12, 12]          12,352
|    └─ReLU: 2-2                         [-1, 64, 12, 12]          --
|    └─Conv2d: 2-3                       [-1, 96, 12, 12]          18,528
|    └─ReLU: 2-4                         [-1, 96, 12, 12]         